In [2]:
import pandas as pd
import datetime
r1=pd.read_csv('NDVI Values at 3 sites - ne1_NDVI.csv')
r2=pd.read_csv('NDVI Values at 3 sites - ne2_NDVI.csv')
r3=pd.read_csv('NDVI Values at 3 sites - ne3_NDVI.csv')
#arm=pd.read_csv('ARM_NDVI.csv')
#arm.drop('Unnamed: 0',axis=1,inplace=True)
#arm.loc[336:343,'2002_NDVI']=None
#arm

## Data Processing functions

In [1]:
import pandas as pd
import datetime
def doy(r):
    return datetime.datetime(int(r.Date[:4]), int(r.Date[5:7]), int(r.Date[8:10])).timetuple().tm_yday
def ameriflux_dataprocess(filename,saveas):
    r=pd.read_csv(filename,skiprows=2).iloc[:,[0,1,3]]
    r.iloc[:,0:2]=r.applymap(str).iloc[:,0:2]
    r['Year'] = r.TIMESTAMP_START.str[:4]
    r['Date'] = r.TIMESTAMP_START.str[:4]+"-"+r.TIMESTAMP_START.str[4:6]+"-"+r.TIMESTAMP_START.str[6:8]
    r['DOY'] = pd.Series.to_frame(r.apply(doy,axis=1))
    r['TA'][r['TA']==-9999.0]=None
    
    gb = r.groupby(['Year','Date'])
    tmax=gb.agg({'Year' : np.min, 'Date':np.min,'DOY':np.min,'TA' : np.max})
    tmin=gb.agg({'Year' : np.min,'Date':np.min,'DOY':np.min,'TA' : np.min})

    tmax.index=range(len(tmax))
    tmin.index=range(len(tmin))

    temp=gb.agg({'Year' : np.min, 'Date':np.min,'DOY':np.min,'TA' : {'tmin' : np.min, 'tmax' : np.max}})
    temp.index=range(len(temp))
    
    temp=pd.concat([temp,temp.TA],axis=1)
    temp.columns=pd.MultiIndex.from_tuples(list(temp.columns)[:-2]+([("TA_soy","tmin"),("TA_soy","tmax")]))
    
    temp.TA=temp.TA.clip(8.0,30.0)
    temp.TA_soy=temp.TA_soy.clip(10.0,25.0)
    temp['tavg_corn']=0.5*(temp.TA.tmin+temp.TA.tmax)
    temp['tavg_soy']=0.5*(temp.TA_soy.tmin+temp.TA_soy.tmax)
    temp['gdd_corn']=temp['tavg_corn']-8
    temp['gdd_soy']=temp['tavg_soy']-10
    temp.to_pickle(saveas)
    return temp
def prism_dataprocess(filename,saveas):
    pr=pd.read_csv(filename, skiprows=10)
    pr['tmin']=5*(pr['tmin (degrees F)']-32)/9
    pr['tmax']=5*(pr['tmax (degrees F)']-32)/9
    df = pr.drop('tmin (degrees F)', 1)
    df = pr.drop('tmax (degrees F)', 1)
    pr['Year']=pr['Date'].str[:4]
    pr['DOY']=pd.Series.to_frame(pr.apply(doy,axis=1))
    pr=pr[['Year','Date','DOY','tmin','tmax']]
    pr.columns=pd.MultiIndex.from_tuples([('Year','amin'),('Date','amin'),('DOY','amin'),('TA','tmin'),('TA','tmax')])
    pr=pd.concat([pr,pr.TA],axis=1)
    pr.columns=pd.MultiIndex.from_tuples(list(pr.columns)[:-2]+([("TA_soy","tmin"),("TA_soy","tmax")]))
    
    pr.TA=pr.TA.clip(8.0,30.0)
    pr.TA_soy=pr.TA_soy.clip(10.0,25.0)
    pr['tavg_corn']=0.5*(pr.TA.tmin+pr.TA.tmax)
    pr['tavg_soy']=0.5*(pr.TA_soy.tmin+pr.TA_soy.tmax)
    pr['gdd_corn']=pr['tavg_corn']-8
    pr['gdd_soy']=pr['tavg_soy']-10
    pr.to_pickle(saveas)
    return pr
def hybrid_dataprocess(ameriflux_picklepath,prism_picklepath,saveas):
    af=pd.read_pickle(ameriflux_picklepath)
    pr=pd.read_pickle(prism_picklepath)
    null_indices=af.TA.tmin.isnull()
    indices=af[null_indices].index
    af.loc[indices, 'gdd_corn'] = list(pr.loc[indices, 'gdd_corn'])
    af.loc[indices, 'gdd_soy'] = list(pr.loc[indices, 'gdd_soy'])
    af.to_pickle(saveas)
    return af

In [52]:
df1 = r1.drop('doy', 1)
df2 = r2.drop('doy', 1)
df3 = r3.drop('doy', 1)
df_arm=arm.drop('doy',1)

In [53]:
import os
import pdb

import numpy as np
import pandas as pd
from scipy.optimize import leastsq


def model_fourier(params, agdd, n_harm):
    """
    Fourier model
    :param params:
    :param agdd:
    :param n_harm:
    :return:
    """
    integration_time = len(agdd)
    t = np.arange(1, integration_time + 1)
    result = t*.0 + params[0]
    w = 1

    for i in range(1, n_harm * 4, 4):
        result = result + params[i] * np.cos(2.0 * np.pi * w * t / integration_time + params[i+1]) \
                 + params[i+2]*np.sin(2.0 * np.pi * w * t / integration_time + params[i+3])
        w += 1

    return result


def mismatch_function(params, func_phenology, ndvi, agdd):
    """
    The NDVI/Phenology model mismatch function
    :param params:
    :param func_phenology:
    :param ndvi:
    :param agdd:
    :param years:
    :return:
    """
    # output stores the predictions
    output = []

    oot = ndvi - func_phenology(params, agdd, n_harm=8)

    [output.append(x) for x in oot]

    return np.array(output).squeeze()


def do_fourier(ndvi, gdd, n_harm=8, init_params=None):
    """
    :param ndvi:
    :param gdd:
    :param n_harm:
    :param init_params:
    :return:
    """
    n_params = 1 + n_harm * 4

    if init_params is None:
        init_params = [.25, ] * n_params
        (xsol, mesg) = leastsq(mismatch_function, init_params, args=(model_fourier, ndvi, gdd), maxfev=1000000)
        model_fitted = model_fourier(xsol, gdd, n_harm)

    return model_fitted


def get_PTD(df):
    """
    Get phenological transition dates (greenup, senescence)
    :param df:
    :return:
    """
    # Input dataframe has an index comprised of day of year and remaining columns signify NDVI
    # Linearly interpolate dataframe columns to fill in missing values
    df = df.apply(pd.Series.interpolate)

    # Now compute mean of all columns and get the smoothened NDVI
    arr_smooth = do_fourier(df.mean(axis=1), [8.0] * len(df))

    # For all other crops and regions, take differential
    # To get doy_green, find the last occurrence of the max differential
    diff_green = np.diff(arr_smooth[:365 + 1])
    doy_green = np.where(diff_green == diff_green.max())[0][-1]
    doy_senesc = np.diff(arr_smooth[:365 + 1]).argmin()

    return doy_green, doy_senesc

In [54]:
print(get_PTD(df1))
print(get_PTD(df2))
print(get_PTD(df3))
print(get_PTD(df_arm.iloc[:180]))

(168, 262)
(170, 264)
(174, 261)
(42, 136)


In [55]:
def discretize(df,site=''):
    if site=='arm':
        doys=169
    else:
        doys=361
    for i in range(0,doys,8):
        #print(pd.Series.to_frame(df1_temp.iloc[i,:]))
        te=pd.DataFrame.transpose(pd.Series.to_frame(df.iloc[i,:]))
        df.iloc[i+1:i+8,:]=te
    return df
df1=discretize(df1)
df2=discretize(df2)
df3=discretize(df3)
df_arm=discretize(df_arm.iloc[:169],'arm')
print(get_PTD(df_arm))

C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(39, 131)


In [56]:
df_arm

,2002_NDVI,2003_NDVI,2004_NDVI
0,6337.733082,5374.032674,4707.574305
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,5547.775947,5020.123424,4937.426210
9,NaN,NaN,NaN


In [276]:
g1=list()
s1=list()
g=0
s=0
for i in range(len(df1.columns)):
    temp=df1.iloc[:,i]
    temp.iloc[0]=temp.
    
    iloc[temp.first_valid_index()]
    g,s=get_PTD(pd.Series.to_frame(temp))
    g1.append(g)
    s1.append(s)
print(g1)
print(s1)

[174, 167, 167, 166, 166, 159, 156, 162, 157, 157, 176, 153, 161, 170, 164, 172]
[264, 261, 267, 263, 255, 253, 257, 269, 262, 251, 282, 233, 263, 261, 260, 256]


In [277]:
g2=list()
s2=list()
for i in range(len(df2.columns)):
    temp=df2.iloc[:,i]
    temp.iloc[0]=temp.iloc[temp.first_valid_index()]
    g,s=get_PTD(pd.Series.to_frame(temp))
    g2.append(g)
    s2.append(s)
print(g2)
print(s2)

[172, 184, 164, 190, 161, 183, 160, 184, 158, 165, 171, 152, 162, 186, 162, 181]
[268, 261, 268, 267, 253, 259, 252, 262, 262, 253, 272, 244, 268, 263, 257, 269]


In [278]:
g3=list()
s3=list()
for i in range(len(df1.columns)):
    temp=df3.iloc[:,i]
    temp.iloc[0]=temp.iloc[temp.first_valid_index()]
    g,s=get_PTD(pd.Series.to_frame(temp))
    g3.append(g)
    s3.append(s)
print(g3)
print(s3)

[174, 98, 163, 186, 160, 183, 160, 187, 158, 188, 170, 173, 164, 189, 167, 180]
[274, 258, 232, 258, 256, 259, 247, 257, 244, 260, 251, 254, 245, 263, 256, 271]


In [57]:
g4=list()
s4=list()
g=0
s=0
for i in range(len(df_arm.columns)):
    temp=df_arm.iloc[:,i]
    temp.iloc[0]=temp.iloc[temp.first_valid_index()]
    g,s=get_PTD(pd.Series.to_frame(temp))
    g4.append(g)
    s4.append(s)
print(g4)
print(s4)

[167, 99, 38]
[127, 131, 24]


## Testing GDD 

In [4]:
def gdd_evaluate(l1,l2,temp_data,crop):
    years=list(map(str, list(range(2001,2016))))
    gdd=list()
    gdds=list()
    precip=list()
    for i in range(len(years)):
            yr=years[i]
            start_doy=l1[i]
            stop_doy=l2[i]
            t=temp_data[temp_data.Year.amin==yr]
            if(crop[i]==1):
                g=t.gdd_corn[start_doy-1:stop_doy].sum()
                gs=t.gdds_corn[start_doy-1:stop_doy].sum()
            else:
                g=t.gdd_soy[start_doy-1:stop_doy].sum()
                gs=t.gdds_soy[start_doy-1:stop_doy].sum()
            p=t.P[start_doy-1:stop_doy].sum()
            precip.append(p[0])
            gdd.append(g)
            gdds.append(gs)
            print(gs)
    print()
    return (gdd,precip)

# for NE1 site

In [3]:
##The Following are the crop and year specific greenup and sen values after downscaling the modis bands at ne1,2 and 3 mead sites 
g1=[174,169,171,172,164,164,156,163,161,167,169,155,167,173,167]
s1=[278,266,271,259,246,263,257,273,266,253,267,237,252,265,262]

g2=[176,186,167,193,161,183,165,190,162,169,175,156,163,189,164]
s2=[275,264,275,271,257,260,256,267,270,257,270,247,275,266,265]
g3=[176,188,167,191,164,185,166,191,161,190,173,179,168,194,169]
s3=[279,262,240,263,251,262,250,261,250,264,253,257,250,266,259]

In [6]:
temp_ne1

Year        Date  DOY   TA         TS         P TA_soy      tavg_corn  \
      amin        amin amin tmin tmax tsmin tsmax sum   tmin tmax             
0     2001  2001-01-01    1  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
1     2001  2001-01-02    2  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
2     2001  2001-01-03    3  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
3     2001  2001-01-04    4  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4     2001  2001-01-05    5  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
5     2001  2001-01-06    6  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
6     2001  2001-01-07    7  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
7     2001  2001-01-08    8  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
8     2001  2001-01-09    9  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
9     2001  2001-01-10   10  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
10    2001  2001-01-11   11  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
11    2001  2001-01-12   12  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
12    2001  2001-01-13   13  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
13    2001  2001-01-14   14  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
14    2001  2001-01-15   15  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
15    2001  2001-01-16   16  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
16    2001  2001-01-17   17  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
17    2001  2001-01-18   18  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
18    2001  2001-01-19   19  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
19    2001  2001-01-20   20  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
20    2001  2001-01-21   21  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
21    2001  2001-01-22   22  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
22    2001  2001-01-23   23  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
23    2001  2001-01-24   24  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
24    2001  2001-01-25   25  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
25    2001  2001-01-26   26  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
26    2001  2001-01-27   27  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
27    2001  2001-01-28   28  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
28    2001  2001-01-29   29  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
29    2001  2001-01-30   30  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
...    ...         ...  ...  ...  ...   ...   ...  ..    ...  ...       ...   
4718  2013  2013-12-02  336  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4719  2013  2013-12-03  337  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4720  2013  2013-12-04  338  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4721  2013  2013-12-05  339  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4722  2013  2013-12-06  340  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4723  2013  2013-12-07  341  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4724  2013  2013-12-08  342  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4725  2013  2013-12-09  343  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4726  2013  2013-12-10  344  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4727  2013  2013-12-11  345  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4728  2013  2013-12-12  346  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4729  2013  2013-12-13  347  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4730  2013  2013-12-14  348  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4731  2013  2013-12-15  349  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4732  2013  2013-12-16  350  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4733  2013  2013-12-17  351  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4734  2013  2013-12-18  352  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4735  2013  2013-12-19  353  NaN  NaN   NaN   NaN NaN   

In [5]:
p1=[130,129,135,123,124,125,121,119,110,109,138,114,119,111,120] # planting, emerging and harvesting doys
e1=[136,138,147,133,137,136,130,129,125,124,146,122,134,127,132]
h1=[291,308,300,288,286,278,309,322,313,264,299,283,295,301,299]
e1=[147]*15
g1=[155]*15
crop1=[1]*15 # all years Maize
#ameriflux_dataprocess("AMF_US-Ne1_BASE-BADM_7-1/AMF_US-Ne1_BASE_HR_7-1.csv","Temperature_data/ameriflux_temp_data_ne1")
#prism_dataprocess("PRISM_NE1_stable_4km_20010101_20151231_41.1651_-96.4766.csv","Temperature_data/prism_temp_data_ne1")
#hybrid_dataprocess("Temperature_data/ameriflux_temp_data_ne1","Temperature_data/prism_temp_data_ne1","Temperature_data/hybrid_temp_data_ne1")
#temp_ne1=pd.read_pickle("Temperature_data/hybrid_temp_data_ne1")
#temp_ne1=pd.read_pickle("Temperature_data/ameriflux_temp_data_ne1")
temp_ne1=pd.read_pickle("AMF_US-Ne1_BASE-BADM_7-1/precip_temperature_data_ne1")
Gdd_planting,Precip_planting=gdd_evaluate(p1,g1,temp_ne1,crop1)
Gdd_emergence,Precip_emergence=gdd_evaluate(e1,g1,temp_ne1,crop1)
Gdd_harvesting,Precip_harvesting=gdd_evaluate(s1,h1,temp_ne1,crop1)

nan
252.97525
195.91275
316.84
342.948
395.04
427.335
374.63925
482.7355
431.20775
214.74775
514.11775
322.2925
0
0

nan
138.45775
99.925
100.21
98.0325
158.6925
109.3125
122.01425
115.6325
137.14275
121.1275
112.615
66.65
0
0



NameError: name 's1' is not defined

## Testing GDD for NE2 site

In [82]:
p2=[131,140,134,153,122,132,122,135,111,110,138,115,120,140,119] # planting, emerging and harvesting doys
e2=[138,148,145,159,134,143,131,145,126,133,146,123,135,146,132]
h2=[295,280,296,291,290,278,309,282,314,259,299,282,295,283,300]
e2=[]
g2=[]
crop2=[1,2,1,2,1,2,1,2,1,1,1,1,1,2,1] # 1- Maize; 2- soy
for crop in crop2:
    if crop==1:
        e2.append(147)
        g2.append(155)
    else:
        e2.append(159)
        g2.append(179)
#ameriflux_dataprocess("AMF_US-Ne2_BASE-BADM_7-1/AMF_US-Ne2_BASE_HR_7-1.csv","Temperature_data/ameriflux_temp_data_ne2")
#prism_dataprocess("PRISM_NE2_stable_4km_20010101_20151231_41.1649_-96.4701.csv","Temperature_data/prism_temp_data_ne2")
#hybrid_dataprocess("Temperature_data/ameriflux_temp_data_ne2","Temperature_data/prism_temp_data_ne2","Temperature_data/hybrid_temp_data_ne2")

In [83]:
#temp_ne2=pd.read_pickle("Temperature_data/ameriflux_temp_data_ne2")
temp_ne2=pd.read_pickle("AMF_US-Ne2_BASE-BADM_7-1/precip_temperature_data_ne2")
Gdd_planting=gdd_evaluate(p2,g2,temp_ne2,crop2)
Gdd_emergence=gdd_evaluate(e2,g2,temp_ne2,crop2)
Gdd_harvesting=gdd_evaluate(s2,h2,temp_ne2,crop2)

nan
450.033
233.88
270.16025
368.70775
528.935
388.293
386.9515
422.009
323.32525
206.435
526.10275
329.13025
0
0

nan
269.235
115.005
222.50025
107.47025
268.55
99.9075
232.32575
114.3525
110.935
117.4825
119.3975
67.1975
0
0

79.0775
100.31775
102.01525
79.27025
282.165
110.6705
319.55
128.86175
74.4025
28.685
159.82775
279.22525
nan
0
0



# for NE3 site

In [84]:
p3=[134,140,133,153,116,131,122,134,112,139,122,135,133,139,120] # planting, emerging and harvesting doys
e3=[141,148,142,159,131,142,133,145,127,147,133,141,141,146,133]
h3=[302,282,286,284,290,281,304,281,315,279,291,274,294,281,301]
crop3=[1,2]*7+[1]
e3=[]
g3=[]
for crop in crop3:
    if crop==1:
        e3.append(147)
        g3.append(155)
    else:
        e3.append(159)
        g3.append(179)
#ameriflux_dataprocess("AMF_US-Ne3_BASE-BADM_7-1/AMF_US-Ne3_BASE_HR_7-1.csv","Temperature_data/ameriflux_temp_data_ne3")
#prism_dataprocess("PRISM_NE3_stable_4km_20010101_20151231_41.1797_-96.4397.csv","Temperature_data/prism_temp_data_ne3")
#hybrid_dataprocess("Temperature_data/ameriflux_temp_data_ne3","Temperature_data/prism_temp_data_ne3","Temperature_data/hybrid_temp_data_ne3")

In [85]:
#temp_ne3=pd.read_pickle("Temperature_data/ameriflux_temp_data_ne3")
temp_ne3=pd.read_pickle("AMF_US-Ne3_BASE-BADM_7-1/precip_temperature_data_ne3")
Gdd_planting=gdd_evaluate(p3,g3,temp_ne3,crop3)
Gdd_emergence=gdd_evaluate(e3,g3,temp_ne3,crop3)
Gdd_harvesting=gdd_evaluate(s3,h3,temp_ne3,crop3)

nan
441.2075
259.275
285.14525
351.54025
534.47
396.7
447.8225
390.233
461.13
291.57275
481.445
194.015
0
0

nan
272.695
116.43
230.93275
102.7125
266.78
106.3475
237.9175
106.53775
260.6425
103.26775
252.665
59.3675
0
0

75.155
148.965
410.80525
98.81025
366.3025
138.2425
389.60775
177.74025
282.0275
103.11775
282.6605
122.59775
nan
0
0



In [8]:
import pandas as pd
t=pd.read_pickle("Temperature_data/prism_temp_data_ne1")
t.to_csv("prism_temp_data_ne1.csv")

In [229]:
t.loc[t['Year']=="2002"]

,Date,tmin (degrees F),tmax (degrees F),tmin,tmax,Year,DOY,tavg,gdd
365,2002-01-01,1.1,18.7,8.0,8.000000,2002,1,8.000000,0.000000
366,2002-01-02,2.4,16.9,8.0,8.000000,2002,2,8.000000,0.000000
367,2002-01-03,2.4,19.0,8.0,8.000000,2002,3,8.000000,0.000000
368,2002-01-04,4.7,34.3,8.0,8.000000,2002,4,8.000000,0.000000
369,2002-01-05,21.0,41.7,8.0,8.000000,2002,5,8.000000,0.000000
370,2002-01-06,32.3,48.1,8.0,8.944444,2002,6,8.472222,0.472222
371,2002-01-07,7.1,33.2,8.0,8.000000,2002,7,8.000000,0.000000
372,2002-01-08,8.5,41.0,8.0,8.000000,2002,8,8.000000,0.000000
373,2002-01-09,29.1,63.7,8.0,17.611111,2002,9,12.805556,4.805556
374,2002-01-10,31.6,54.3,8.0,12.388889,2002,10,10.194444,2.194444


In [50]:
temp_ne2.columns

MultiIndex(levels=[['DOY', 'Date', 'P', 'TA', 'TA_soy', 'Year', 'tavg_corn', 'tavg_soy', 'gdd_corn', 'gdd_soy'], ['amin', 'sum', 'tmax', 'tmin', '']],
           labels=[[5, 1, 0, 3, 3, 2, 4, 4, 6, 7, 8, 9], [0, 0, 0, 3, 2, 1, 3, 2, 4, 4, 4, 4]])

In [79]:
temp_ne1[(temp_ne1['Year']['amin']=='2003')&(temp_ne1['DOY']['amin']>=147)]

Year        Date  DOY     TA              P TA_soy        tavg_corn  \
      amin        amin amin   tmin   tmax     sum   tmin   tmax             
876   2003  2003-05-27  147  10.29  28.28   0.000  10.29  25.00    19.285   
877   2003  2003-05-28  148  16.86  29.53   0.000  16.86  25.00    23.195   
878   2003  2003-05-29  149  11.76  30.00   6.096  11.76  25.00    20.880   
879   2003  2003-05-30  150  15.38  30.00   0.000  15.38  25.00    22.690   
880   2003  2003-05-31  151  12.27  23.50   0.000  12.27  23.50    17.885   
881   2003  2003-06-01  152  14.44  25.33   5.080  14.44  25.00    19.885   
882   2003  2003-06-02  153  10.65  14.42   7.620  10.65  14.42    12.535   
883   2003  2003-06-03  154  10.40  17.04   0.000  10.40  17.04    13.720   
884   2003  2003-06-04  155   9.24  23.06   0.000  10.00  23.06    16.150   
885   2003  2003-06-05  156  11.61  24.04   8.890  11.61  24.04    17.825   
886   2003  2003-06-06  157  11.65  23.90   0.000  11.65  23.90    17.775   
887   2003  2003-06-07  158  10.88  15.38  14.478  10.88  15.38    13.130   
888   2003  2003-06-08  159  11.79  21.76   0.000  11.79  21.76    16.775   
889   2003  2003-06-09  160  11.45  30.00   5.080  11.45  25.00    20.725   
890   2003  2003-06-10  161  15.00  23.62   0.000  15.00  23.62    19.310   
891   2003  2003-06-11  162  12.90  27.28   0.000  12.90  25.00    20.090   
892   2003  2003-06-12  163  16.60  20.89   8.890  16.60  20.89    18.745   
893   2003  2003-06-13  164  14.43  29.46   0.000  14.43  25.00    21.945   
894   2003  2003-06-14  165  15.81  30.00   0.000  15.81  25.00    22.905   
895   2003  2003-06-15  166  16.55  30.00   0.000  16.55  25.00    23.275   
896   2003  2003-06-16  167  17.17  30.00   0.000  17.17  25.00    23.585   
897   2003  2003-06-17  168  17.62  30.00   0.000  17.62  25.00    23.810   
898   2003  2003-06-18  169  18.27  28.21  33.786  18.27  25.00    23.240   
899   2003  2003-06-19  170  15.69  28.72   0.000  15.69  25.00    22.205   
900   2003  2003-06-20  171  14.13  28.49   0.000  14.13  25.00    21.310   
901   2003  2003-06-21  172  19.24  29.86   0.254  19.24  25.00    24.550   
902   2003  2003-06-22  173  21.09  30.00   5.588  21.09  25.00    25.545   
903   2003  2003-06-23  174  19.43  30.00   0.254  19.43  25.00    24.715   
904   2003  2003-06-24  175  24.10  30.00   5.588  24.10  25.00    27.050   
905   2003  2003-06-25  176  12.87  27.19   0.254  12.87  25.00    20.030   
...    ...         ...  ...    ...    ...     ...    ...    ...       ...   
1065  2003  2003-12-02  336   8.00   8.00   5.080  10.00  10.00     8.000   
1066  2003  2003-12-03  337   8.00   8.00   0.762  10.00  10.00     8.000   
1067  2003  2003-12-04  338   8.00   8.00   1.016  10.00  10.00     8.000   
1068  2003  2003-12-05  339   8.00   8.00   0.000  10.00  10.00     8.000   
1069  2003  2003-12-06  340   8.00   8.00   0.000  10.00  10.00     8.000   
1070  2003  2003-12-07  341   8.00   8.00   0.000  10.00  10.00     8.000   
1071  2003  2003-12-08  342   8.00   8.00   0.000  10.00  10.00     8.000   
1072  2003  2003-12-09  343   8.00   8.00   0.000  10.00  10.00     8.000   
1073  2003  2003-12-10  344   8.00   8.00   0.000  10.00  10.00     8.000   
1074  2003  2003-12-11  345   8.00   8.00   0.000  10.00  10.00     8.000   
1075  2003  2003-12-12  346   8.00   8.00   0.000  10.00  10.00     8.000   
1076  2003  2003-12-13  347   8.00   8.00   0.000  10.00  10.00     8.000   
1077  2003  2003-12-14  348   8.00   8.00   0.000  10.00  10.00     8.000   
1078  2003  2003-12-15  349   8.00   8.00   0.254  10.00  10.00     8.000   
1079  2003  2003-12-16  350   8.00   8.00   0.000  10.00  10.00     8.000   
1080  2003  2003-12-17  351   8.00   8.00   0.508  10.00  10.00     8.000   
1081  2003  2003-12-18  352   8.00   8.00   0.000  10.00  10.00     8.000   
1082  2003  2003-12-19  353   8.00   8.00   0.000  10.00  10.00     8.000   
1083  2003  2003-12-20  354   8.00   8.00   0.000  10.00  10.00     8.000   
10

In [60]:
type(temp_ne2.Year.amin=='2001')

pandas.core.series.Series